In [12]:
!pip install ipywidgets
!pip install opencv-python numpy matplotlib ipywidgets

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output
import imageio
from io import BytesIO

# Helper function to convert image for saving to GIF
def image_to_rgb(img):
    if len(img.shape) == 2:  # Grayscale
        return cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Helper function to save matplotlib figure to BytesIO
def fig_to_image():
    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img = imageio.imread(buf)
    buf.close()
    plt.close()
    return img

# Helper function to resize image to common size
def resize_image(img, size=(600, 600)):
    return cv2.resize(img, size, interpolation=cv2.INTER_AREA)

# Helper function to create image with title
def create_titled_image(img, title):
    plt.figure(figsize=(6, 6))
    plt.imshow(image_to_rgb(img))
    plt.title(title)
    plt.axis('off')
    img = fig_to_image()
    return resize_image(img)

# Helper function to plot histogram
def plot_histogram(img, title, channels=[0], color='gray'):
    plt.figure(figsize=(6, 6))  # Changed to square for consistency
    for i, col in enumerate(channels):
        hist = cv2.calcHist([img], [i], None, [256], [0, 256])
        plt.plot(hist, color=color if len(channels) == 1 else ['r', 'g', 'b'][i], label=['R', 'G', 'B'][i])
    plt.title(title)
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.legend()
    img = fig_to_image()
    return resize_image(img)

# Load image
image_path = 'sample_image.jpg'  # Replace with your image path
img = cv2.imread(image_path)
if img is None:
    raise ValueError("Image not found. Please provide a valid image path.")

# List to store all images for GIF
gif_images = []

# 1. Original image
gif_images.append((create_titled_image(img, "Original Image"), "Original Image"))

# 2. RGB channels
r, g, b = cv2.split(img)
gif_images.append((create_titled_image(r, "Red Channel"), "Red Channel"))
gif_images.append((create_titled_image(g, "Green Channel"), "Green Channel"))
gif_images.append((create_titled_image(b, "Blue Channel"), "Blue Channel"))

# 3. HSV channels
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(img_hsv)
gif_images.append((create_titled_image(h, "Hue Channel"), "Hue Channel"))
gif_images.append((create_titled_image(s, "Saturation Channel"), "Saturation Channel"))
gif_images.append((create_titled_image(v, "Value Channel"), "Value Channel"))

# 4. Modify regions
# Rectangle modified
img_rect = img.copy()
img_rect[50:150, 50:150] = [0, 0, 255]  # BGR: Red
gif_images.append((create_titled_image(img_rect, "Rectangle Modified"), "Rectangle Modified"))

# Region replaced
img_replace = img.copy()
img_replace[50:150, 200:300] = img[200:300, 50:150]
gif_images.append((create_titled_image(img_replace, "Region Replaced"), "Region Replaced"))

# 5. Brightness and contrast adjustments with histograms
# Original image and histogram
gif_images.append((create_titled_image(img, "Original Image"), "Original Image"))
gif_images.append((plot_histogram(img_gray := cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), "Original Grayscale Histogram"), "Original Histogram"))

# Manual adjustment
alpha = 1.5
beta = 50
img_manual = np.clip(alpha * img.astype(float) + beta, 0, 255).astype(np.uint8)
gif_images.append((create_titled_image(img_manual, "Manual Brightness/Contrast"), "Manual Brightness/Contrast"))
gif_images.append((plot_histogram(cv2.cvtColor(img_manual, cv2.COLOR_BGR2GRAY), "Manual Adjustment Histogram"), "Manual Histogram"))

# Auto adjustment
img_auto = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
gif_images.append((create_titled_image(img_auto, "Auto Brightness/Contrast"), "Auto Brightness/Contrast"))
gif_images.append((plot_histogram(cv2.cvtColor(img_auto, cv2.COLOR_BGR2GRAY), "Auto Adjustment Histogram"), "Auto Histogram"))

# Create single GIF
imageio.mimsave('image_processing.gif', [img for img, _ in gif_images], duration=1000)

# 6. Bonus: Interactive brightness and contrast adjustment (Colab-compatible)
def interactive_adjustment(alpha=1.0, beta=0):
    clear_output(wait=True)
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    plt.figure(figsize=(6, 6))
    plt.imshow(image_to_rgb(adjusted))
    plt.title(f"Interactive Adjustment (Alpha={alpha:.2f}, Beta={beta})")
    plt.axis('off')
    plt.savefig('interactive_adjustment.png')  # Save final interactive state
    plt.show()

# Create sliders
alpha_slider = widgets.FloatSlider(value=1.0, min=0.0, max=3.0, step=0.1, description='Alpha (Contrast):')
beta_slider = widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='Beta (Brightness):')

# Link sliders to function
widgets.interact(interactive_adjustment, alpha=alpha_slider, beta=beta_slider)

<ipython-input-11-ee616eef8bbd>:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(buf)


interactive(children=(FloatSlider(value=1.0, description='Alpha (Contrast):', max=3.0), FloatSlider(value=0.0,…

<function __main__.interactive_adjustment(alpha=1.0, beta=0)>